In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress




In [3]:
#read csv files
populationcsv = "UGB/Top100_CitiesPop.csv"
population = pd.read_csv(populationcsv)
population.head()


,ID,Rank,City,2010 Census,2011_Estimate,2012_Estimate,2013_Estimate,2014_Estimate,2015_Estimate,2016_Estimate,2017_Estimate,2018_Estimate
0,3651000,1,New York City,8175133,8272963,8348032,8398739,8437387,8468181,8475976,8438271,8398748
1,644000,2,Los Angeles,3792621,3821136,3852532,3883916,3913260,3943215,3969262,3982002,3990456
2,1714000,3,Chicago,2695598,2708209,2719735,2726772,2728524,2726215,2718946,2713067,2705994
3,4835000,4,Houston,2099451,2124143,2160086,2198280,2240982,2286630,2309752,2317445,2325502
4,455000,5,Phoenix,1445632,1470052,1499839,1527336,1556552,1584927,1613581,1634984,1660272


In [4]:
densitycsv = "UGB/top_100.csv"
density = pd.read_csv(densitycsv)
density.head()

,city,state,county,pop 2010,pop_density
0,New York City,NY,New York,8537673,13401.8
1,Los Angeles,CA,Los Angeles,12815475,3295.0
2,Chicago,IL,Cook,8675982,4612.0
3,Houston,TX,Harris,5446468,1401.0
4,Phoenix,AZ,Maricopa,4081849,1212.0


In [22]:
popdens = pd.merge(population, density, how="left", left_on="City", right_on="city")
popdens_clean = popdens[["Rank", "City", "2010 Census", "state", "pop_density"]]
popdens_clean = popdens_clean.rename(columns = {"pop_density": "Persons by km2", "state": "State"})
popdens_clean
output_file = "top_100/ugb.csv"

In [24]:
popdens_clean.to_csv("top_100clean.csv", encoding="utf-8", index=False)